## Working with ISEA4T in Vgrid DGGS (Windows only)

[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.gishub.vn/lab/index.html?path=notebooks/vgrid/05_isea4t.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeoshub/vgrid/blob/master)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeoshub/vgrid/HEAD)

For working with Vgrid DGGS directly in GeoPandas and Pandas, please use [vgridpandas](https://pypi.org/project/vgridpandas/), [vgridpandas document](https://vgridpandas.gishub.vn)

### Install vgrid
Uncomment the following line to install [vgrid](https://pypi.org/project/vgrid/) if needed.

In [ ]:
# %pip install vgrid --upgrade

### latlon2isea4t

In [ ]:
from vgrid.conversion.latlon2dggs import latlon2isea4t

lat = 10.775276
lon = 106.706797
res = 12
isea4t_id = latlon2isea4t(lat, lon, res)
isea4t_id

### ISEA4T to Shapely Polygons

In [ ]:
from vgrid.conversion.dggs2geo.isea4t2geo  import isea4t2geo

isea4t_geo = isea4t2geo(isea4t_id)
isea4t_geo

### ISEA4T to GeoJSON

In [ ]:
from vgrid.conversion.dggs2geo.isea4t2geo import isea4t2geojson

isea4t_geojson = isea4t2geojson(isea4t_id)
isea4t_geojson

### Vector to ISEA4T

In [ ]:
# %pip install folium

In [ ]:
from vgrid.conversion.vector2dggs.vector2isea4t import vector2isea4t

file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/shape/polygon.geojson"
)
isea4t_geojson = vector2isea4t(file_path, resolution=16, compact=False, predicate = "intersects")
# Change compact to True to see the compact version of the rHEALPix cells

# Visualize the output GeoJSON
import folium

m = folium.Map(tiles="CartoDB positron")

isea4t_layer = folium.GeoJson(
isea4t_geojson,
style_function=lambda x: {
    "fillColor": "blue",
    "color": "black",
    "fillOpacity": 0.3,
    "weight": 1,
},
tooltip=folium.GeoJsonTooltip(
    fields=["isea4t", "resolution", "cell_area"],
    aliases=["ISEA4T ID", "Resolution", "Area (m²)"],
    style="""
        background-color: white;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
),
).add_to(m)

m.fit_bounds(isea4t_layer.get_bounds())

# Display the map
m

### ISEA4T Compact

In [ ]:
from vgrid.conversion.dggscompact import isea4tcompact

isea4t_compacted = isea4tcompact(
    isea4t_geojson, isea4t_id="isea4t"
)

# Visualize the output GeoJSON
import folium

m = folium.Map(tiles="CartoDB positron")

isea4t_compacted_layer = folium.GeoJson(
    isea4t_compacted,
    style_function=lambda x: {
        "fillColor": "blue",
        "color": "black",
        "fillOpacity": 0.3,
        "weight": 1,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["isea4t", "resolution", "cell_area"],
        aliases=["ISEA4T ID", "Resolution", "Area (m²)"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(isea4t_compacted_layer.get_bounds())

# Display the map
m

### ISEA4T Expand

In [ ]:
from vgrid.conversion.dggscompact import isea4texpand

isea4t_expanded = isea4texpand(
    isea4t_geojson, resolution=17
)

# Visualize the output GeoJSON
import folium

m = folium.Map(tiles="CartoDB positron")

isea4t_expanded_layer = folium.GeoJson(
    isea4t_expanded,
    style_function=lambda x: {
        "fillColor": "blue",
        "color": "black",
        "fillOpacity": 0.3,
        "weight": 1,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["isea4t", "resolution", "cell_area"],
        aliases=["ISEA4T ID", "Resolution", "Area (m²)"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(isea4t_expanded_layer.get_bounds())

# Display the map
m

### ISEA4T Binning

In [ ]:
from vgrid.binning.isea4tbin import isea4tbin
import geopandas as gpd
file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/csv/dist1_pois.csv"
)
isea4t_bin = isea4tbin(file_path, resolution=15, stats="count")
isea4t_bin